__Assignment 5__

5) Calculate the average monthly precipitation for the Östergotland region (list of stations is provided in the separate file) for the period 1993-2016. In order to do this, you will first need to calculate the total monthly precipitation for each station before calculating the monthly average (by averaging over stations).

In this exercise you will use the *precipitation-readings.csv* and *stations-Ostergotland.csv* files. 

__HINT__ (not for the SparkSQL lab): Avoid using joins here! 
*stations-Ostergotland.csv* is small and if distributed will cause a number of unnecessary shuffles when joined with precipitation RDD. If you distribute *precipitation-readings.csv* then either repartition your stations RDD to 1 partition or make use of the collect to acquire a python list and broadcast function to broadcast the list to all nodes.

The output should contain the following information:
__Year, month, average monthly precipitation__


For each question include the following data in the report and sort it as shown:
- year, month, avgMonthlyPrecipitation ORDER BY year DESC, month DESC

In [1]:
# create the setup
from pyspark import SparkContext
from pyspark.sql import SQLContext, Row  
from pyspark.sql import functions as F
sc =SparkContext()
sqlContext=SQLContext(sc)

In [20]:
from pyspark.sql.functions import col, round, upper

In [2]:
# path to temperature-readings-tiny.csv
path_unifolder_precip_tiny = "/Users/phillipholscher/OneDrive - Linköpings universitet/LiU/Statistics and Machine Learning/Spring19T2/Big Data Analytics/Labs/Lab 2/precipitation-readings-tiny.csv"
path_unifolder_stations = "/Users/phillipholscher/OneDrive - Linköpings universitet/LiU/Statistics and Machine Learning/Spring19T2/Big Data Analytics/Labs/Lab 2/stations-Ostergotland.csv"


In [7]:
# read and create rdd, this needs to be tranfsormed to data frame later on
precip = sc.textFile(path_unifolder_precip_tiny)
parts_precip = precip.map(lambda l:l.split(";"))
precipReadings = parts_precip.map(lambda p: Row(station=p[0],  date=p[1], year=p[1].split("-")[0], month=p[1].split("-")[1], day=p[1].split("-")[2], time=p[2],  Precip=float(p[3]), quality=p[4]))
precipReadings.take(10)


[Row(Precip=0.0, date='1995-08-01', day='01', month='08', quality='Y', station='103100', time='00:00:00', year='1995'),
 Row(Precip=0.0, date='1995-08-01', day='01', month='08', quality='Y', station='103100', time='01:00:00', year='1995'),
 Row(Precip=0.0, date='1995-08-01', day='01', month='08', quality='Y', station='103100', time='02:00:00', year='1995'),
 Row(Precip=0.0, date='1995-08-01', day='01', month='08', quality='Y', station='103100', time='03:00:00', year='1995'),
 Row(Precip=0.0, date='1995-08-01', day='01', month='08', quality='Y', station='103100', time='04:00:00', year='1995'),
 Row(Precip=0.0, date='1995-08-01', day='01', month='08', quality='Y', station='103100', time='05:00:00', year='1995'),
 Row(Precip=0.0, date='1995-08-01', day='01', month='08', quality='Y', station='103100', time='06:00:00', year='1995'),
 Row(Precip=0.0, date='1995-08-01', day='01', month='08', quality='Y', station='103100', time='07:00:00', year='1995'),
 Row(Precip=0.0, date='1995-08-01', day=

In [8]:
# create a data frame from rdd
schemaPrecipReadings =  sqlContext.createDataFrame(precipReadings)
schemaPrecipReadings.registerTempTable("precipReadings")
schemaPrecipReadings.show(10)

+------+----------+---+-----+-------+-------+--------+----+
|Precip|      date|day|month|quality|station|    time|year|
+------+----------+---+-----+-------+-------+--------+----+
|   0.0|1995-08-01| 01|   08|      Y| 103100|00:00:00|1995|
|   0.0|1995-08-01| 01|   08|      Y| 103100|01:00:00|1995|
|   0.0|1995-08-01| 01|   08|      Y| 103100|02:00:00|1995|
|   0.0|1995-08-01| 01|   08|      Y| 103100|03:00:00|1995|
|   0.0|1995-08-01| 01|   08|      Y| 103100|04:00:00|1995|
|   0.0|1995-08-01| 01|   08|      Y| 103100|05:00:00|1995|
|   0.0|1995-08-01| 01|   08|      Y| 103100|06:00:00|1995|
|   0.0|1995-08-01| 01|   08|      Y| 103100|07:00:00|1995|
|   0.0|1995-08-01| 01|   08|      Y| 103100|08:00:00|1995|
|   0.0|1995-08-01| 01|   08|      Y| 103100|09:00:00|1995|
+------+----------+---+-----+-------+-------+--------+----+
only showing top 10 rows



In [36]:
# read and create rdd, this needs to be tranfsormed to data frame later on
stations = sc.textFile(path_unifolder_stations)
# create DataFrame from RDD
parts_stations = stations.map(lambda a: a.split(';'))
schemaStationsOestergotland = parts_stations.map(lambda x: Row(station = x[0], name = x[1]))
schemaStationsOestergotland = sqlContext.createDataFrame(schemaStationsOestergotland)

schemaStationsOestergotland.show(10)

+-----------+-------+
|       name|station|
+-----------+-------+
|    Örberga|  84260|
|  Västra Ny|  85630|
|Kettstaka A|  85460|
|   Godegård|  85450|
| Zinkgruvan|  85490|
| Kärnskogen|  85650|
|   Kvarn Mo|  85390|
|   Finspång|  85410|
| Simonstorp|  86470|
|Stavsjö Aut|  86440|
+-----------+-------+
only showing top 10 rows



In [37]:
# filter for given time interval
schemaPrecipReadings_filtered = schemaPrecipReadings\
.select(["station", "year", "month","Precip"])\
.filter((schemaPrecipReadings_filtered["year"] >= 1993) & 
       (schemaPrecipReadings_filtered["year"] <= 2016))

schemaPrecipReadings_filtered.show(10)

+-------+----+-----+------+
|station|year|month|Precip|
+-------+----+-----+------+
| 103100|1995|   08|   0.0|
| 103100|1995|   08|   0.0|
| 103100|1995|   08|   0.0|
| 103100|1995|   08|   0.0|
| 103100|1995|   08|   0.0|
| 103100|1995|   08|   0.0|
| 103100|1995|   08|   0.0|
| 103100|1995|   08|   0.0|
| 103100|1995|   08|   0.0|
| 103100|1995|   08|   0.0|
+-------+----+-----+------+
only showing top 10 rows



In [ ]:
"""
Do not run this cell - the result would be empty because I use the tiny data set
"""
# filter for Östergotland stations
#schemaPrecipReadings_filtered = schemaPrecipReadings_filtered.join(schemaStationsOestergotland, 'station', 'inner')

#schemaPrecipReadings_filtered.show(10)

In [38]:
# compute the avg monthly precip for Östergotland stations
# but first we need to comput the total monthy precip for each station
schemaPrecipReadings_total_monthly_precip = schemaPrecipReadings_filtered\
.groupBy("station", "year", "month").agg(F.sum("Precip").alias("Precip_month_sum"))

schemaPrecipReadings_total_monthly_precip.show(10)


+-------+----+-----+------------------+
|station|year|month|  Precip_month_sum|
+-------+----+-----+------------------+
| 103100|1996|   07| 39.10000000000002|
| 103100|1996|   03|4.1000000000000005|
| 103100|1995|   10|              51.3|
| 103100|1996|   10| 33.90000000000002|
| 103100|1996|   04|24.700000000000006|
| 103100|1996|   02|               4.6|
| 103100|1995|   12| 9.599999999999998|
| 103100|1996|   01| 5.999999999999999|
| 103100|1995|   11| 33.49999999999999|
| 103100|1995|   08|              36.5|
+-------+----+-----+------------------+
only showing top 10 rows



In [39]:
# compute the avg monthly precip for Östergotland stations
schemaPrecipReadings_avg_monthly_precip = schemaPrecipReadings_total_monthly_precip\
.groupBy("year", "month").agg(round(F.avg("Precip_month_sum"),2).alias("avg_monthly_precip"))\
.orderBy(["year", "month"], ascending = False)

schemaPrecipReadings_avg_monthly_precip.show(10)

+----+-----+------------------+
|year|month|avg_monthly_precip|
+----+-----+------------------+
|1996|   10|              33.9|
|1996|   09|              48.2|
|1996|   08|              41.0|
|1996|   07|              39.1|
|1996|   06|              59.7|
|1996|   05|              62.3|
|1996|   04|              24.7|
|1996|   03|               4.1|
|1996|   02|               4.6|
|1996|   01|               6.0|
+----+-----+------------------+
only showing top 10 rows

